## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saskylakh,RU,71.9167,114.0833,14.00,86,93,1.68,overcast clouds
1,1,El Vigia,VE,8.6219,-71.6506,74.82,95,100,3.24,overcast clouds
2,2,Avarua,CK,-21.2078,-159.7750,77.05,88,98,1.14,overcast clouds
3,3,Ushtobe,KZ,45.2514,77.9838,58.60,53,100,23.53,moderate rain
4,4,Bluff,NZ,-46.6000,168.3333,53.80,68,94,21.47,overcast clouds


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would prefer for your trip?"))
max_temp = float(input("What is the maximum temperature you would prefer for your trip?"))

What is the minimum temperature you would prefer for your trip?60
What is the maximum temperature you would prefer for your trip?75


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] < max_temp)]
filtered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,El Vigia,VE,8.6219,-71.6506,74.82,95,100,3.24,overcast clouds
7,7,Kintampo,GH,8.0563,-1.7306,74.28,76,100,3.02,overcast clouds
8,8,Alta Floresta,BR,-9.8756,-56.0861,71.58,96,50,3.29,scattered clouds
14,14,Katsuura,JP,35.1333,140.3000,69.80,70,47,6.78,scattered clouds
15,15,Diamantina,BR,-18.2494,-43.6003,60.39,94,6,7.40,clear sky


In [26]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.count()

City_ID                200
City                   200
Country                200
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_cities_df = filtered_cities_df.dropna()
filtered_cities_df.count()

City_ID                200
City                   200
Country                200
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,El Vigia,VE,74.82,overcast clouds,8.6219,-71.6506,
7,Kintampo,GH,74.28,overcast clouds,8.0563,-1.7306,
8,Alta Floresta,BR,71.58,scattered clouds,-9.8756,-56.0861,
14,Katsuura,JP,69.80,scattered clouds,35.1333,140.3000,
15,Diamantina,BR,60.39,clear sky,-18.2494,-43.6003,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found .... skipping.")
        

Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.
Hotel not found .... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != '')]
clean_hotel_df.count()

City                   182
Country                182
Max Temp               182
Current Description    182
Lat                    182
Lng                    182
Hotel Name             182
dtype: int64

In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))